## YouTube-Transcribe

In [2]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

### 1. Load YouTube transcript

In [4]:
url = "https://www.youtube.com/watch?v=9-Jl0dxWQs8"
loader = YoutubeLoader.from_youtube_url(url)
docs = loader.load()

### 2. Split into chunks

In [5]:

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(docs)

### 3. Create embeddings + vector store

In [6]:

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=os.getenv("GOOGLE_API_KEY"))

vectorstore = FAISS.from_documents(docs, embeddings)

### 4. Create retriever

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    api_key=os.getenv("GOOGLE_API_KEY")
)

### 5. Build Q&A Chain

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

### 6. Ask about the video

In [14]:
query = "What is the main idea discussed in this video?"
response = qa.run(query)
print(response)

The main idea discussed in this video is about how neural networks, specifically in the context of natural language processing, store and process information, particularly factual knowledge. The video uses a toy example of storing the fact that "Michael Jordan plays basketball" to illustrate these concepts.

Key points include:

*   **High-dimensional space for representation:** The network maps inputs into a higher-dimensional space where different concepts or features can be represented by directions.
*   **Linear operations and non-linearity:** Initial operations are linear, but a non-linear function is applied to introduce a "yes" or "no" type of decision, which is crucial for complex tasks like understanding language.
*   **Superposition and interpretability:** The video touches upon the concept of "superposition," a hypothesis suggesting that features might be represented in a way that allows for more information to be stored in the high-dimensional space than if features were st